# Convolutional Neural Network model - an introduction (Part 1)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/paolodeangelis/Sistemi_a_combustione/blob/main/3.1-CNN_Intro_P1.ipynb)

![working progress](https://raw.githubusercontent.com/paolodeangelis/Sistemi_a_combustione/main/assets/img/warning-work-in-progress.jpg)